[도서]  [데이터실무분석 with파이썬 예제](https://wikibook.co.kr/playwithdata/)

# 4.1 인스타그램 크롤링 

### (1) selenium 설치 
   
    + pip install selenium
    
    + 웹드라이버 다운로드
        http://chromedriver.chromium.org/downloads
        chromedirver_win32.zip 파일 다운로드 받고 압축풀기

        설치된 크롬 버전과 맞는 버전을 다운로드 해야 한다.
        
        
---        
[참고] 사이트 변경으로 인해 데이타를 못 가져올 수도 있다        

### 4.1.2 인스타그램 검색 결과 URL을 만들어서 접속하기

In [40]:
# 예제 4-1 인스타그램 검색결과 URL을 만드는 함수
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
    

In [3]:
# !pip install selenium

In [9]:
# 예제 4-2 selenium으로 URL 접속하기
from selenium import webdriver

# 본인의 webdriver 설치 경로 맞추기
driver = webdriver.Chrome("D:/python/eAnalysis/13.big/5-1_sungsu_crawling_map/webdriver/chromedriver.exe")   

word = "성수동맛집"
url = insta_searching(word)
driver.get(url)


### 4.1.3 첫 번째 게시글 열기 

In [41]:
# 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    
    driver.implicitly_wait(10) # 로딩 속도 때문에 클릭이벤트를 못하기에 추가 ( 상황에 따라 실행되었다 안되었다 함 )
    
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

### 4.1.4 게시글 정보 가져오기 

In [42]:
# 예제 4-4 게시글 정보 가져오기
import re

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data


### 4.1.5 다음 게시글 열기

In [43]:
# 예제 4-5 다음 게시글 열기
def move_next(driver):

    driver.implicitly_wait(10) # 로딩 속도 때문에 클릭이벤트를 못하기에 추가 ( 상황에 따라 실행되었다 안되었다 함 )
    
    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)


In [64]:
from selenium.webdriver.common.keys import Keys

id ='heyj1@naver.com'
pword ='wjddkdls12'

def login(driver):
    driver.implicitly_wait(3) # 암묵적으로 자원로드될 때까지 3초 기다림
    driver.find_element_by_name('username').send_keys(id)
    driver.find_element_by_name('password').send_keys(pword)
    right = driver.find_element_by_css_selector ('body > div.RnEpo._Yhr4 > div.pbNvD.fPMEg._8PWHW > div > div.Igw0E.IwRSH.eGOV_._4EzTm.MGdpg.lDRO1 > div > div > div.gr27e.o7laV > div > form > div:nth-child(4) > button > div')
    right.click()
    later_btn = driver.find_element_by_css_selector ('#react-root > section > main > div > div > div > div > button')
    later_btn.click()
    time.sleep(3)

### 4.1.6 여러 게시글 정보 수집하기

In [109]:
# 예제 4-6 인스타그램 크롤링
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re


# ① 크롬 브라우저 열기
driver = webdriver.Chrome("D:/python/eAnalysis/13.big/5-1_sungsu_crawling_map/webdriver/chromedriver.exe") 

# ② 인스타그램 검색페이지 URL 만들기
word = "성수동맛집"    #검색어
url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(3)



# ④ 첫 번째 게시글 열기
select_first(driver)
login(driver)
select_first(driver)
# ⑤ 비어있는 변수(results)만들기
results = [ ]


# ⑥→⑦→⑧ 여러 게시물 수집하기
# target = int(driver.find_element_by_css_selector('#react-root > section > main > header > div.WSpok > div.f7QXd.HfISj > div.Igw0E.IwRSH.eGOV_._4EzTm.a39_R > span > span').text.replace(',',''))
target = 300
# 크롤링할 게시글 수 : 우선은 5개 하고 실제적으로 50개
for i in range(target):
    data = get_content(driver)    # 게시글 정보 가져오기
    results.append(data)
    move_next(driver)
    time.sleep(2)

print(results[:2])

[['흐아...시원한 콩국수 땡기는날🔥콩국수는 무조건 굵은소금인데 스토리답장에 설탕,흑설탕 추천하시는분이 은근 계셔서 깜놀🤷🏻\u200d♂️#성수동맛집 #성수맛집 #조계현칼국수', '2020-07-03', '3,398', '성수동', ['#성수동맛집', '#성수맛집', '#조계현칼국수']], ['먹고싶었던 초당옥수수랑 신비복숭아 샀다! 신난다💞', '2020-07-02', '1,250', '', []]]


In [110]:
# 예제 4-7 크롤링 결과 저장하기

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./files/sungsudong.MatJip.xlsx')


### 4.1.8 여러 엑셀 파일의 중복을 제거한 후 통합 저장

인스타그램에서 여러 개의 키워드로 검색하거나 어러 번에 걸쳐 데이터를 수집하는 경우가 많다.

동일한 게시물을 중복으로 수집되는 경우 중복을 제거해야 한다.

In [103]:
# 예제 4-8 여러 개의 저장파일 통합하기
jeju_insta_df = pd.DataFrame( [ ] )

folder = './files/'

# f_list = ['3_1_crawling_jejudoMatJip.xlsx', '3_1_crawling_jejudoGwanGwang.xlsx', '3_1_crawling_jejuMatJip.xlsx', '3_1_crawling_jejuYeoHang.xlsx']
f_list = ['sungsu.xlsx', 'sungsu.MatJip.xlsx','sungsu.date.xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns =['no','content','data','like','place','tags']

# ???? jeju_insta_df.columns  왜 7개 이지???

In [105]:
# 예제 4-9 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)
jeju_insta_df.to_excel('./files/sungsu.crawling.raw.xlsx', index = False)